In [1]:
from mip import Model, xsum, maximize, BINARY, minimize, INTEGER
import networkx as nx
import pickle
from itertools import product
import numpy as np
import pandas as pd

In [2]:
g = nx.read_gml('networks/italy.gml', label="id")

g.edges # ezek a linkek, melyik tornybol melyik toronyba megy a link - ezek ugymond a linkek nevei

#ez lesz a H0 vektor, nem kell vektort elég ha csak egyszerüen egy szám
#Hnull = np.ones((len(g.edges))) * 6
Hnull = 6

# ezek a cut SLRG-k vagyis a link csoportok 
with open ('min_cut_SRLGs/italy', 'rb') as fp:
    all_cut_srlgs = pickle.load(fp)

cut_srlgs = [all_cut_srlgs[2]]
# ezek az intezitások, l,p,m
intensity = np.load('intensities/italy.npy')
#data[0][0]  ez az első link első poziciohoz tartozo földrendés intenzitásai


df = pd.read_csv('earthquake_probabilities/italy.csv')
df.insert(loc=0, column='Epicenter', value=list(zip(df.Lat, df.Long)))
df.set_index('Epicenter', inplace=True)
df.drop(['Lat', 'Long'], axis=1, inplace=True)

prob_matrix = df.to_numpy()

epicenters = df.index.to_numpy()
magnitudes = df.columns.to_numpy()

# ez ezeknek a hossza
L = len(g.edges)
P = len(epicenters)
M = len(magnitudes)
S = len(cut_srlgs)

# treschold cons 3-hoz
T = 0.01

##
cost = 1

In [3]:
a = intensity[14]
b = intensity[18]
c = intensity[26]

columns = ((a>6) & (b>6) & (c>6)).any(axis=0)
rows = ((a>6) & (b>6) & (c>6)).any(axis=1)

intensity = intensity[:,:,columns][:,rows]
prob_matrix = prob_matrix[:,columns][rows]

P,M = prob_matrix.shape

epicenters = range(P)
magnitudes = range(M)




In [4]:
intensity.shape

(35, 8889, 21)

In [4]:
m = Model("fradir")

#decision variable
deltaH = [m.add_var(var_type=INTEGER, lb=0, ub=6) for l,_ in enumerate(g.edges)] # 0 tol 6 ig terjedhet, csak integer lehet 
Z = [m.add_var(var_type=BINARY) for z in range(S* P * M)]
Y = [m.add_var(var_type=BINARY) for y in  range(L* P * M)]
print("Dec varok megvannak")

#objective
m.objective = minimize(xsum(cost * deltaH[l] for l,_ in enumerate(g.edges)))
print("Objective is megvan")

#cons1
for l,_ in enumerate(g.edges):
    m += xsum( intensity[l,p,m] * Y[l*P*M+p*M+m] for p,_ in enumerate(epicenters) for m,_ in enumerate(magnitudes)) >= (Hnull + deltaH[l])
    print(l)
print("cons1 megvan")

#cons2
for (c,s), (p,_), (m,_) in product(*[enumerate(cut_srlgs), enumerate(epicenters),enumerate(magnitudes)]):
    m += len(s) * Z[c*P*M+p*M+m] == xsum(Y[list(g.edges).index(linkID)*P*M+p*M+m] for linkID in s)
print("cons2 megvan")

#cons3
for c,s in enumerate(cut_srlgs):
    m += xsum( Z[c*P*M+p*M+m] * prob_matrix[p,m] for p,_ in enumerate(epicenters) for m,_ in enumerate(magnitudes) ) <= T
print("cons3 megvan")
m.optimize()

selected = [deltaH[l] for l,_ in enumerate(g.edges) if deltaH[l] >= 0.99]
print("selected items: {}".format(selected))

gurobi version 9.0 found
Dec varok megvannak
Objective is megvan
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
cons1 megvan
cons2 megvan


DeprecationWarning: Inplace operations are deprecated

In [41]:
Y = [m.add_var(var_type=BINARY) for l,_ in enumerate(g.edges) for p,_ in enumerate(epicenters[:2]) for M,_ in enumerate(magnitudes[:2])]




SyntaxError: invalid syntax (<ipython-input-41-56f4a8a715ed>, line 1)

# ---------------------------------------------------------------

In [7]:
def ff(a,b):
    return a*2 + b**2

In [14]:
#koltseg =  [11, 3,3, 6, 1, 9, 16,78]
koltseg = 8
fejlesztes = [17,6,4,61,16,42,156, 7]

y = [1,1,0,1,0,1,0,0]
z = [0,1,1,1,0,1,0,1]

items = len(fejlesztes)

m = Model("fradir")

#decision variable
x = [m.add_var(var_type=BINARY) for i in range(items)]
fejlesztes_Dvar = [m.add_var(var_type=INTEGER, lb=1, ub=6) for i in range(items)]

#objective
m.objective = maximize(xsum(koltseg * fejlesztes[i] * x[i] for i in range(items)))

#cons
#m += xsum(y[i] * x[i] for i in range(items)) >= 2

#m += xsum(z[i] * x[i] for i in range(items)) >= 2

#m += xsum(((y[i] + z[i]) % 2) * x[i]  for i in range(items)) == 0

for i in range(items):
    m += 10 * x[i] <= fejlesztes[i]

#m += xsum(( fejlesztes[i] * x[i]  for i in range(items))) <= 14

m += xsum(x[i] for i in range(items)) == 2


m.optimize()

selected = [i for i in range(items) if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [3, 6]


In [14]:
ff(3,4)

22

# GEKKO

In [6]:
import random
from gekko import GEKKO
import numpy as np

In [7]:
koltseg =  [11, 3,3, 6, 1, 9, 16,78]
fejlesztes = [17,67,4,61,16,42,156, 7]

y = [1,0,1,1,0,1,1,0]
z = [0,1,1,1,0,1,1,1]

prob = [0.5, 1.2, 1.5, 1.7, 0.1, 1.8, 0.4, 1.9]

items = len(koltseg)

# Create model
m = GEKKO()

# Variables
x = m.Array(m.Var,items,lb=0,ub=1,integer=True)
fejlesztes_Dvar = m.Array(m.Var, items, lb=1,ub=6, integer=True)
b = m.Array(m.Var, items, lb=1,ub=8, integer=True)
#x2 = m.Array(m.Var, len(w),lb=0,ub=1,integer = True)

# Objective
m.Maximize(sum(koltseg[i] * fejlesztes_Dvar[i] * x[i] for i in range(items) ))

# Constraint
m.Equation(sum([(y[i] + z[i]) * x[i] for i in range(items)]) == 2)

m.Equation(sum([x[i] for i in range(items)]) == 1)

m.Equation(sum((z[i] * prob[i]) *x[i] for i in range(items)) <=1.6 )

# Optimize with APOPT
m.options.SOLVER = 1

m.solve(disp = False)

# Print the value of the variables at the optimum
print(x)
for i in range(len(x)):
    if x[i][0] == 1.0:
        print("Sorszam: {}., koltseg: {}, fejlesztes: {}".format(i,koltseg[i], fejlesztes_Dvar[i][0]))

[[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [1.0] [0.0]]
Sorszam: 6., koltseg: 16, fejlesztes: 6.0


In [2]:
data = np.load('italy.npy')

In [13]:
pd.DataFrame(data[0]).describe()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
count,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,...,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000
mean,1.154713,1.174508,1.196344,1.220365,1.246715,1.275495,1.306710,1.340423,1.376788,1.415944,...,3.194826,3.325022,3.459114,3.597036,3.738632,3.883800,4.032280,4.183553,4.337154,4.492752
std,0.578387,0.617518,0.658248,0.700550,0.744388,0.789728,0.836545,0.884773,0.934307,0.985046,...,2.112041,2.145883,2.176857,2.204735,2.229386,2.250615,2.268428,2.283199,2.295290,2.304934
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.260501,1.422601,1.584701,1.746801,1.908901,2.071001,2.233101,2.395201,2.557301,2.719401
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.691067,2.853167,3.015267,3.177367,3.339467,3.501567,3.663667,3.825767,3.987867,4.149967
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.004125,...,4.570325,4.732425,4.894525,5.056625,5.218725,5.380825,5.542925,5.705025,5.867125,6.029225
max,6.113586,6.275686,6.437786,6.599886,6.761986,6.924086,7.086186,7.248286,7.410386,7.572486,...,11.138686,11.300786,11.462886,11.624986,11.787086,11.949186,12.111286,12.273386,12.435486,12.597586
